In [93]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import time
import requests
from deepface import DeepFace
import traceback

In [82]:
# Get the user ID from the review html

def get_user_id(review):
    user_url = soup.select("a[href^='/user_details']")[0]['href']
    user_id = user_url[user_url.find('userid=')+len('userid='):]
    user_base_url = 'https://www.yelp.com/user_details?userid='
    user_url = '{0}{1}'.format(user_base_url, user_id)

def get_url(url):
    req = Request(url)
    while True:
        try:
            html_page = urlopen(req) 
            break
        # TODO: Fix
        except requests.exceptions.HTTPError as err:
            print(err.response.status_code)
            print(err.response.text)
            time.sleep(10)
            pass
    return html_page

def argmax(d):
    h_k = None
    h_v = 0
    for k, v in d.items():
        if v > h_v:
            h_k = k
            h_v = v
            #print(f'highest_v: {h_v}')
    return h_k

In [40]:
get_url('https://www.gmail.com')

In [41]:
get_url('https://www.yelp.com/biz/china-palace-ypsilanti-2?start=20')

In [127]:
page_count = 0
running_asian_star_total = 0
count_asians = 0
desired_asians = 10

example_restaurant_url = 'https://www.yelp.com/biz/tk-wu-ann-arbor'

# Get total star rating
req = Request(example_restaurant_url)
html_page = urlopen(req)
soup = BeautifulSoup(html_page, 'html.parser')

# ANNOYING- why is it that there's a space before 'five-stars' to find it if using Chrome, but not if using selector?
overall_star_rating = float(
    soup.select("div[data-testid='review-summary'] div[class^='five-stars']")[0]['aria-label'][:-len(' star rating')]
)
overall_star_rating

3.5

In [128]:
restaurant_url

'https://www.yelp.com/biz/china-palace-ypsilanti-2/?start=60&sort_by=date_desc'

In [129]:
results = []

# Get Asian star rating
while count_asians < desired_asians:
    if page_count>0:
        restaurant_url = '{0}/?start={1}&sort_by=date_desc'.format(example_restaurant_url, page_count*10)
    else:
        restaurant_url = '{0}/?sort_by=date_desc'.format(example_restaurant_url, page_count*10)

    print(f'url: {restaurant_url}')

    # Get reviews
    html_page = get_url(restaurant_url)
    soup = BeautifulSoup(html_page, 'html.parser')
    reviews = soup.select("div[class^='review']")

    for review in reviews:
        try:
            # Get user photo url
            user_url = review.select("a[href^='/user_details']")[0]['href']
            user_id = user_url[user_url.find('userid=')+len('userid='):]
            user_base_url = 'https://www.yelp.com/user_details?userid='
            user_url = '{0}{1}'.format(user_base_url, user_id)

            print('looking at user id {0}'.format(user_id))

            # Get user photo
            html_page = get_url(user_url)
            soup = BeautifulSoup(html_page, 'html.parser')
            user_photo_url = soup.select('div.user-profile_avatar img')[0]['src']

            # Guess user race
            guessed_race = None
            try: 
                guessed_race_dict = DeepFace.analyze(
                    img_path = user_photo_url, 
                    actions = ['race']
                )[0]['race']
                guessed_race = argmax(guessed_race_dict)
                print('user is predicted {0}'.format(guessed_race))
            except ValueError as e:
                print(e)
                
            # Get review star rating
            star_rating = int(
                review.select("div[class^='five-stars']")[0]['aria-label'][0]
            )
            
            # TODO: Get text here
            
            
            # Append results
            results.append({
                'restaurant_url': restaurant_url,
                'user_url': user_url,
                'guessed_race': guessed_race,
                'star_rating': star_rating,
                #'text':  TODO
            })

            if guessed_race and guessed_race=="asian":
                count_asians += 1

                running_asian_star_total += star_rating
                print(f'{count_asians} Asians found with running rating of {running_asian_star_total/count_asians}/5')

                time.sleep(1)
            
        except Exception as e:
            print(e)
            print(traceback.format_exc())
            pass

    page_count += 1

asian_average = running_asian_star_total / count_asians

url: https://www.yelp.com/biz/tk-wu-ann-arbor/?sort_by=date_desc
looking at user id z-F1v_u4_gStaw6R0AgNnA
Face could not be detected. Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
looking at user id 175uYPDHoOdYMIhSi_j9-Q
Face could not be detected. Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
looking at user id G_1BJlqGvGDlEvyzPNERmg
Face could not be detected. Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
looking at user id LP9E_9T0dH9WP76EJ7eUvg


Action: race: 100%|██████████| 1/1 [00:00<00:00,  3.72it/s]


user is predicted asian
1 Asians found with running rating of 5/5
looking at user id nSt9bq9-aFhOwOdFgJciMA


Action: race: 100%|██████████| 1/1 [00:00<00:00,  3.51it/s]


user is predicted asian
2 Asians found with running rating of 9/5
looking at user id lfefbHZT2REBX1VHv6QB7g
Face could not be detected. Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
looking at user id DvJ7XddXlBTsi7Zro6x32A
Face could not be detected. Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
looking at user id gwXGqUGR70FSN94zLoMHOQ
Face could not be detected. Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
looking at user id HElEfBFwzSs67EyQNpiOYg
Face could not be detected. Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
looking at user id dgYqFfJspVivXP3tHoj6eg
Face could not be detected. Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
url: https://www.yelp.com/biz/tk-wu-ann-arbor/?start=10&sort_by=date_desc
looki

...

user is predicted asian
10 Asians found with running rating of 41/5
looking at user id PUkF_AHx0PpRJW6GQZBchg


Action: race: 100%|██████████| 1/1 [00:00<00:00,  3.73it/s]


user is predicted indian
looking at user id UPC4orX2XMblQosGrH-nwQ


Action: race: 100%|██████████| 1/1 [00:00<00:00,  3.51it/s]


user is predicted asian
11 Asians found with running rating of 45/5
looking at user id 5Xrxvvyn9HOaM_dqiP6MZw
Face could not be detected. Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
looking at user id AS0uNwHwECs7NpXroM5WJw
Face could not be detected. Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
looking at user id nK1cZXx2BMjwOo49lOdH7g
Face could not be detected. Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
looking at user id o8MJUnLKYCV1RJPtPEAHMw
Face could not be detected. Please confirm that the picture is a face photo or consider to set enforce_detection param to False.


In [99]:
print('Overall rating: {0}'.format(overall_star_rating))
print('Asian rating: {0:.1f}'.format(asian_average))

Overall rating: 3.5
Asian rating: 3.8


In [121]:
results[0]

{'restaurant_url': 'https://www.yelp.com/biz/china-palace-ypsilanti-2/?sort_by=date_desc',
 'user_url': 'https://www.yelp.com/user_details?userid=tVaRKAMi70Ir44L5Y9l1HA',
 'guessed_race': None,
 'star_rating': 1}

In [122]:
results_df = pd.DataFrame(results)
results_df.head()

,restaurant_url,user_url,guessed_race,star_rating
0,https://www.yelp.com/biz/china-palace-ypsilant...,https://www.yelp.com/user_details?userid=tVaRK...,None,1
1,https://www.yelp.com/biz/china-palace-ypsilant...,https://www.yelp.com/user_details?userid=N_LXF...,None,1
2,https://www.yelp.com/biz/china-palace-ypsilant...,https://www.yelp.com/user_details?userid=bA6ad...,asian,4
3,https://www.yelp.com/biz/china-palace-ypsilant...,https://www.yelp.com/user_details?userid=UrdLZ...,None,4
4,https://www.yelp.com/biz/china-palace-ypsilant...,https://www.yelp.com/user_details?userid=wQubH...,asian,2


In [123]:
results_df[results_df.guessed_race=='asian']['star_rating'].mean()

3.1666666666666665

In [124]:
results_df[results_df.guessed_race!='asian']['star_rating'].mean()

2.6052631578947367

In [125]:
results_df['star_rating'].mean()

2.74

In [126]:
results_df.to_csv('results.csv', mode='a', index=False, header=False)